# Baruch IMG Portfolio Management Risk

In [76]:
# Data manipulation 
import numpy as np
import pandas as pd
import time as t

# Plotting
import seaborn as sns
import matplotlib.pyplot
%matplotlib inline 

# Stock pulling  
from alpha_vantage.timeseries import TimeSeries

For more information on the alpha_vantage api or the python wrapper feel free to read the documentation provided
- AV API Docs https://www.alphavantage.co/documentation/
- Python Wrapper by Romel https://github.com/RomelTorres/alpha_vantage

**NOTE that AV API limits call requests to 5 a minute**

In [20]:
api_key = "JAFMAHL3Q9GK8XC1" #Alpha vantage api-key recieived throguh the service

## Risk Management Helper Functions 

In [63]:
# data columns present from timeseries pull by Alpha Vantage API
data_columns = ['1. open', '2. high', '3. low', '4. close', '5. adjusted close', 
                '6. volume', '7. dividend amount', '8. split coefficient']

In [77]:
def stock_puller(ticker_list: list, col: str, outputsize: str = 'compact'):
    """
    :param ticker_list: a list of tickers for data to be extracted eg: ['SPY', 'GOOG', 'AAPL']
    :param col: provided string to extract the desired column type, illustrated in column_desc list
    :param outputsize: 'compact' = 100 most recent, 'full' = historical security list
    :return: returns a dataframe object with relevant timeseries data (index by date, column ticker symbols)
    """
    big_dict = {}
    ts = TimeSeries(key=api_key, output_format='pandas', retries=10) # function call from Python wrapper to extract timeseries 
    
    for ticker in ticker_list:
        # iterates through the security list provided and extracts relevant timeseries information
        data = ts.get_daily_adjusted(symbol=ticker, outputsize=outputsize)[0][col]

        big_dict[ticker] = data.tolist()  # dictionary storing security time series for faster concatination 
        t.sleep(12)
        
    big_df = pd.DataFrame.from_dict(data=big_dict)  # returns large dataframe for list of securities
    return big_df

In [78]:
def log_returns(price_df: pd.DataFrame):
    """
    Extracts the log returns for relevant price data
    :param price_df: a dataframe object storing price values for a given underlying (index by date, column ticker symbols)
    :return: a dataframe for log returns of select securities
    """
    big_dict = {}

    for ticker in price_df.columns:
        # iterates through the security list provided and extract provided values via dict manipulation
        security_list = price_df[ticker]
        log = np.log(security_list.values) - np.log(np.roll(security_list.values, 1))  # computes log return 
        big_dict[ticker] = log[1:].tolist() # eliminates the first NaN row and returns list from ndarray

    big_df = pd.DataFrame.from_dict(data=big_dict) # dataframe index by data column all ticker symbols
    return big_df

In [79]:
def linear_returns(price_df: pd.DataFrame):
    """
    Extracts the linear returns for relevant price data 
    :param price_df: a dataframe object storing price values for a given underlying (index by date, column ticker symbols)
    :return: a dataframe for linear returns of select securities
    """
    big_dict = {}

    for ticker in price_df.columns:
        # iterates through the security list provided and extract provided values via dict manipulation
        linear_ret = price_df[ticker].pct_change() # pct_change pandas function for linear returns
        big_dict[ticker] = linear_ret[1:].tolist() # eliminates the first NaN row and returns list from ndarray 

    big_df = pd.DataFrame.from_dict(data=big_dict) # dataframe index by data column all ticker symbols
    return big_df

In [100]:
def volatility(price_df: pd.DataFrame, rolling_window: int = 10):
    """
    Extracts the standard deviation of relevant price data
    :param price_df: a dataframe object storing price values for a given underlying (index by date, column ticker symbols)
    :return: a dataframe for linear returns of select securities
    """
    log_ret = log_returns(price_df)
    
    big_dict = {}

    for ticker in price_df.columns:
        # iterates through the security list provided and extract provided values via dict manipulation
        volatility = log_ret[ticker].rolling(rolling_window).std() # rolling standard deviation pandas function for log returns
        big_dict[ticker] = volatility[rolling_window-1:].tolist() # eliminates the NaN rows and returns list from ndarray 

    big_df = pd.DataFrame.from_dict(data=big_dict) # dataframe index by data column all ticker symbols
    return big_df

In [81]:
def google_api():
    return 1

## RIsk Management Models

In [57]:
def VaR():
    return 1

In [58]:
def CVaR():
    return 1

Let $r$ be the (random) return of a portfolio and $r_M$ the return of the market portfolio at investment horizon. The portfolio's $\beta$ is defined by the ratio 

$$
\beta := \frac{cov(r, r_M)}{\sigma_M^2} = \rho \frac{\sigma}{\sigma_M},
$$

where $\rho$ is the correlation coefficient between $r$ and $r_M$. 

In particular, if $r_n$ is the linear return of the risky asset $n$, then $\beta_n := \frac{cov(r_n, r_M)}{\sigma_M^2}$ is the beta of risky asset $n$.

In [61]:
def Beta():
    return 1

## Optimization Models

In [59]:
def MSR():
    return 1

### Minimum variance portfolio (MVP)

The weights $w_{mvp}$ for minimum variance portfolio is determined by the solution to the constrained optimization problem 

$$
\min_{w} w^{T} \Sigma w
$$

subject to 

$$
w^{T}u = 1.
$$

By applying the method of Lagrange multiplier, we obtain the solution as 

$$
w_{mvp} = \frac{\Sigma^{-1} u}{u' \Sigma^{-1} u}.
$$

The expected return $\ell_{mvp}$ and the variance $\sigma_{mvp}^2$ of the minimum variance portfolio are given by 

\begin{eqnarray*}
&& \ell_{mvp} = w_{mvp}' \, \ell = \frac{u' \Sigma^{-1}\ell}{u^{T} \Sigma^{-1}u}, \\
&& \sigma_{mvp}^2 = w_{mvp}' \, \Sigma \, w_{mvp} = \frac1{u^{T} \Sigma^{-1}u}. 
\end{eqnarray*}

In [60]:
def MVP():
    return 1

### Effiecient Frontier Curve
For a given expected return $\mu$, determine the weights of the portfolio that has the smallest variance. This is equivalent to
solving the constrained optimization problem

$$
\min_{w} w' \Sigma w 
$$

subject to

$$
u^{T}w = 1, \quad w'\ell = \mu.$$

By applying the method of Lagrange multiplier, the solution to the optimization problem is given by

$$
w_{eff}(\mu) = \frac{B\Sigma^{-1}u - A\Sigma^{-1}\ell}D + \frac{C \Sigma^{-1}\ell - A\Sigma^{-1}u}D \mu
$$

where
$$
\begin{eqnarray*}
&& A = u' \Sigma \ell, \qquad 
B = \ell' \Sigma \ell, \\
&& C = u' \Sigma u, \qquad 
D = B C - A^2. 
\end{eqnarray*}$$

In [30]:
def eff(cov: np.matrix, mu: np.array, target: float):
    """
    Given target return, determine the weights of the portfolio with the smallest variance
    :param cov: provided covariance matrix for the underlying securities (np.matrix object)
    :param mu: expected returns/historical returns for underlying securities (np.array object)
    :param target: target portfolio return for basket of securites (float object)
    :return: 
    """
    assert 0 < target
    
    base = np.ones(len(mu))
    inv = np.linalg.inv(cov)
    
    A = base.dot(cov).dot(mu).item(0)
    B = mu.dot(cov).dot(mu).item(0)
    C = base.dot(cov).dot(base).item(0)
    D = B*C-A**2
    
    left = ((B*inv).dot(base) - (A*inv).dot(mu))/D
    right = target * ((C*inv).dot(mu)-(A*inv).dot(base))/D
    
    return left + right
    

In [54]:
# variable assignment 
mu = np.array([0.3, 0.4, 0.2])
target = 0.45
cov = np.matrix([[0.4, 0.3, 0.1],[0.3, 0.5, 0.2],[0.1, 0.2, 0.1]])

In [55]:
eff(cov, mu, target)

matrix([[ -381.25,  1481.25, -3006.25]])